# Análisis de textos

Empezamos instalando una biblioteca que vamos a utilizar

In [ ]:
import warnings
warnings.filterwarnings('ignore')

bibliotecas = ["spacytextblob","flair","goose3","wordcloud"]

import sys
import os.path
from subprocess import check_call
import importlib
import os
import numpy as np

def instala_bibliotecas(bibliotecas):
  print("Instalando bibliotecas")
  alguna = False
  for m in bibliotecas:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m, " encontrado")
      else:
          print(m, " No encontrado, instalando...",end="")
          try:
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡Hecho!")
            alguna = True
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

  print("¡Terminado!")
  if alguna:
    print("Recuerda reiniciar (Entorno de ejecución + Reiniciar entorno de ejecución) para que la(s) nueva(s) biblioteca(s) estén disponbles")

pipelines = ['en_core_web_sm']
#pipelines = ["en_core_web_lg"]
def instala_pipelines(pipelines):
  print("Instalando pipelines de Spacy")
  for p in pipelines:
      print("Instalando ",p)
      try:
        r = check_call([sys.executable, "-m", "spacy", "download", p])
        print("¡Hecho!")
      except:
        print("¡Problema al instalar ",p,"! ¿seguro que el pipeline existe?",sep="")

  print("¡Terminado!")

instala_pipelines(pipelines)

instala_bibliotecas(bibliotecas)




In [ ]:

import nltk
nltk.download('punkt')

### ¿De dónde salen los datos?



*   Programación con bibliotecas
*   [Programas especializados](https://chrome.google.com/webstore/detail/aliexpress-reviews-export/glnajkhggmbljlcafknlkcihcheoankc?hl=en)
* Bases de datos

In [ ]:
# origen https://www.amazon.com/-/es/product-reviews/B07ZPKF8RG/ref=cm_cr_unknown?ie=UTF8&filterByStar=two_star&reviewerType=all_reviews&pageNumber=1#reviews-filter-bar
textos = ['Broken. It was supposed to be a B-Day gift for my mom and now she doesn’t get a gift on her Bday',
          'Touchscreen became almost completely unresponsive over 20% of surface area within days',
          "A little over 90 days, hardware failure, NO Solution, DON'T BUY",
          "Wanted to love it, but it had too many problems upon arrival",
          "Phone is great but the battery health is 76%",
          "Happy customer! It came with protective glass installed.",
          "Great phone for the price!",
          "I love this phone",
          "Definitely Recommend!"]

Librería Spacy

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')

nlp.add_pipe('spacytextblob')

for text in textos:
    print(text)
    doc = nlp(text) # esto devuelve datos sobre el "sentimiento" del texto
    print(doc._.blob.polarity)
    print("="*30)

Librería Flair

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')

for s in textos:
  text = Sentence(s)

  classifier.predict(text)
  value = text.labels[0].to_dict()
  print(s)
  print(value)
  print("="*30)

Para jugar

In [ ]:
s = "Wonderful room, beautiful views"
text = Sentence(s)
classifier = TextClassifier.load('en-sentiment')
classifier.predict(text)
value = text.labels[0].to_dict()
print(value)

¿Cuál es mejor?  Vamos a utilizar una base de datos de opiniones de películas ya etiquetadas y ver cúal acierta más

In [ ]:
import pandas as pd
df = pd.read_csv("https://github.com/RafaelCaballero/tdm/blob/master/datos/IMDB10K.zip?raw=true",compression='zip')
total = 250
df = df.sample(250)
df

SpaCy

In [ ]:
from tqdm import tqdm


spacyOp = []
for text in tqdm(df.review):
  doc = nlp(text)
  polaridad = doc._.blob.polarity
  spacyOp.append(0 if polaridad<0 else +1)

Flair

In [ ]:
flairOp = []
for s in tqdm(df.review):
    text = Sentence(s)
    classifier.predict(text)
    value = text.labels[0].to_dict()["value"]
    flairOp.append(+1 if value=="POSITIVE" else 0)

Comparamos con el sentimiento "real"

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np


def show_cm(y_real,y_pred):
    labels = list(set(y_real))
    cf_matrix = confusion_matrix(y_real, y_pred, labels=labels)
    ax = sns.heatmap(cf_matrix/sum(cf_matrix), annot=True, fmt='.2', cmap='Blues')
y_real = [+1 if v=="positive" else 0 for v in df.sentiment]

print("Spacy")
show_cm(y_real, spacyOp)


In [ ]:
print("Flair")
show_cm(y_real, flairOp)

Curiosidades....

In [ ]:
frase = "jew"

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')

nlp.add_pipe('spacytextblob')

print("SpaCy")
doc = nlp(frase)
print("Polaridad: ",doc._.blob.polarity)
print("="*30)

print("Flair")
text = Sentence(frase)
classifier.predict(text)
value = text.labels[0].to_dict()
print(value)

## NER (Named Entity Recognition)

Reconocimiento de sustantivos que denotan nombres, organizaciones, etc. Importante para saber de qué se está hablando en textos complejos

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-spanish-large")

In [ ]:


# make example sentence
sentence = Sentence("George Washington fue a Washington, mientras que mi amiga Belén visitaba Belén")

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
s = """
Hermoso y clásico apartamento a cuadras de la Plaza Mayor
10
Liked · Lindo apartamento. Cómodo y espacioso. Muy bien ubicado. Javier fue muy amable en todo. Pasamos una muy grata estancia como familia.
Disliked · La señal Wifi tenía bajo rango, desconectandose cada cierto tiempo
"""


sentence = Sentence(s)

# predict NER tags
tagger.predict(sentence)


# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

## Frases y palabras

Una de las tareas más comunes al tratar con textos largos es separar en frases y mostrar las palabras más frecuentes

In [ ]:
from goose3 import Goose
#import es_core_news_sm

nlp = spacy.load('en_core_web_sm')

# descargamos la noticia
g = Goose()
pag_dir = "https://elpais.com/espana/2023-02-23/espana-sufre-hoy-el-dia-mas-duro-del-nuevo-episodio-invernal-con-11-comunidades-en-alerta-por-nevadas-a-cotas-muy-bajas.html"
articulo = g.extract(url=pag_dir)
texto = articulo.cleaned_text
# utilizamos el modelo para contar oraciones
#nlp = es_core_news_sm.load()
doc = nlp(texto)
print("El artículo incluye ", len(list(doc.sents)),"oraciones")
for i,s in enumerate(doc.sents):
  print(i,s)

Nos preguntamos qué palabras aparecen más en el texto. Una primera versión con no demasiado éxito

In [ ]:
from collections import Counter
palabras = texto.split(" ")
c2 = Counter(palabras)
print(c2)

No resulta muy útil debido a la abundancia de las palabras vacías o "stop words". Veamos cómo quitarlas

In [ ]:
from collections import Counter

nlp = spacy.load('es_core_news_md')

doc = nlp(texto)
palabras = [p.text.upper() for p in doc
             if p.is_alpha and not p.is_stop]
frecuencia = Counter(palabras)
palabras_comunes = frecuencia.most_common(5)
print(palabras_comunes)

## Nubes de palabras

In [ ]:
from goose3 import Goose
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# descargamos la noticia
g = Goose()
pag_dir = "https://elpais.com/diario/1980/12/10/cultura/345250806_850215.html"
#pag_dir = "https://elpais.com/espana/2023-02-23/espana-sufre-hoy-el-dia-mas-duro-del-nuevo-episodio-invernal-con-11-comunidades-en-alerta-por-nevadas-a-cotas-muy-bajas.html"

articulo = g.extract(url=pag_dir)
texto = articulo.cleaned_text

# generar nube de palabras
nlp = spacy.blank('es')
palabrasvacías = nlp.Defaults.stop_words
#palabrasvacías.add("zona")
w  = WordCloud(stopwords = palabrasvacías)
w.generate(texto)

# mostramos el resultado
fig, ax = plt.subplots(1,1, figsize=(20,7),dpi=100)
plt.imshow(w)
fig.savefig("wordcloud1.png")